# ExaGO Jupyter Tutorial

We have a Python wrapper for ExaGO installed in this container through sapck. You can view the Dockerfile in `.devcontainer/Dockerfile`. See the Appendix for more details.

In [1]:
# Did we install correctly?
import exago

In [2]:
# Get pwd and change directory to where datafiles are
import os
print(f'{os.getcwd()}')
os.chdir('../..')
print(f'{os.getcwd()}')

/home/app/docs/devcontainer
/home/app


In [3]:
exago.initialize("opflow")
opf = exago.OPFLOW()
opf.read_mat_power_data('datafiles/case9/case9mod_gen3_wind2.m')
opf.solve()
filepath = './docs/devcontainer/sample_opflow.output'
opf.save_solution(exago.CSV, filepath)
opf.print_solution()
del opf
exago.finalize()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      114
Number of nonzeros in inequality constraint Jacobian.:       72
Number of nonzeros in Lagrangian Hessian.............:       96

Total number of variables............................:       24
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       16
                     variables with only upper bounds:        0
Total number of equality constraints.................:       18
Total number of inequality constra

0

		Optimal Power Flow
Model                               POWER_BALANCE_POLAR
Solver                              IPOPT
Objective                           MIN_GEN_COST
Initialization                      MIDPOINT
Gen. bus voltage mode               VARIABLE_WITHIN_BOUNDS
Load loss allowed                   NO
Power imbalance allowed             NO
Ignore line flow constraints        NO

Number of variables                 24
Number of equality constraints      18
Number of inequality constraints    18

Convergence status                  CONVERGED
Objective value                     2850.19

------------------------------------------------------------------------------------------------------
Bus        Pd      Pdloss Qd      Qdloss Vm      Va      mult_Pmis      mult_Qmis      Pslack         Qslack        
------------------------------------------------------------------------------------------------------
1         0.00    0.00    0.00    0.00   1.040   0.000      2064.25         0.

In [4]:
import csv 
import json

def add_keys(dict, row, tag=""):
    for i in range(0, len(row)):
        if row[i] == "Bus":
            row[i] = "Num"
        elif row[i] == "Gen bus":
            row[i] = "Genbus" 
    for key in row:
        if tag:
            if tag not in dict.keys():
                dict[tag] = {}
            if key == "Bus":
                dict[tag]["Num"] = []
            elif key == "Gen bus":
                dict[tag]["Genbus"] = []
            else:
                dict[tag][key] = [] 
        else:
            dict[key] = 0
    return dict

def write_json(dict):
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(dict, indent=4)
        jsonf.write(jsonString)

def count_lines_in_file(filename):
    lines = 0
    with open(filename, encoding='utf-8') as csvf: 
        lines = sum(1 for _ in csvf)
    csvf.close()
    return lines

def opflow_csv_to_json(csvFilePath, jsonFilePath):
    #count the number of lines in the csv file
    lines = count_lines_in_file(csvFilePath)

    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's reader
        csvReader = csv.reader(csvf) 
        current_row = 0
        temp = {}
        keys = True

        #add first two rows manually
        row = next(csvReader)
        current_row += 1
        temp = add_keys(temp, row)
        current_keys = row
        #check if there's another row
        if current_row < lines:
            row = next(csvReader)
            current_row += 1
        #add values to corresponding keys
        for x in range(0, len(current_keys)):
            if x < 3:
                temp[current_keys[x]] =  int(row[x])
            else:
                temp[current_keys[x]] =  float(row[x])

        #hard coded category names and how many rows
        tags = [("Bus", 9), ("Branch", 9), ("Gen", 3)]
        for pair in tags:
            tag = pair[0]
            num = pair[1]
            #check if there's another row
            if current_row < lines:
                row = next(csvReader)
                current_row += 1
            temp = add_keys(temp, row, tag)
            current_keys = row

            for k in range(0, num): #rows in each category
                #check if there's another row
                if current_row < lines:
                    row = next(csvReader)
                    current_row += 1

                if tag == "Bus":
                   #add values to corresponding keys
                    for x in range(0, len(current_keys)):
                        for j in range(0, len(row)):
                            if j == 0:
                                row[j] = int(row[j])
                            else:
                                row[j] = float(row[j])
                        temp[tag][current_keys[x]].append(row[x])   
                elif tag == "Branch": 
                    #add values to corresponding keys
                    for x in range(0, len(current_keys)):
                        for j in range(0, len(row)):
                            if j >= 0 and j < 3:
                                row[j] = int(row[j])
                            else:
                                row[j] = float(row[j])
                        temp[tag][current_keys[x]].append(row[x])  
                elif tag == "Gen": #cannot cast the string fields
                    #add values to corresponding keys
                    for x in range(0, len(current_keys)):
                        for j in range(0, len(row)):
                            if j >= 0 and j < 2:
                                row[j] = int(row[j])
                            elif j == 2:
                                row[j] = row[2].strip()
                            else:
                                row[j] = float(row[j])
                        temp[tag][current_keys[x]].append(row[x])  
                else:
                    #add values to corresponding keys
                    for x in range(0, len(current_keys)):
                        temp[tag][current_keys[x]].append(float(row[x]))
            
        print(temp)
        write_json(temp) 
        return

#if cell 2 is run, then ./ is the top level of the ExaGO dir
csvFilePath = r'./docs/devcontainer/sample_opflow.output'
jsonFilePath = r'./docs/devcontainer/sample_opflow_data.json'

opflow_csv_to_json(csvFilePath, jsonFilePath)

print(f"data converted from {csvFilePath} to {jsonFilePath}")

{'nbus': 9, 'ngen': 3, 'nbranch': 9, 'baseMVA': 100.0, 'Bus': {'Num': [1, 2, 3, 4, 5, 6, 7, 8, 9], 'Pd': [0.0, 0.0, 0.0, 0.0, 75.0, 90.0, 0.0, 100.0, 0.0], 'Qd': [0.0, 0.0, 0.0, 0.0, 50.0, 30.0, 0.0, 35.0, 0.0], 'Vm': [1.04, 1.02, 1.02, 1.03, 1.01, 1.02, 1.03, 1.02, 1.03], 'Va': [0.0, 4.54, 2.9, -2.19, -3.38, -4.26, 0.75, -1.72, 0.21], 'mult_Pmis': [2064.25, 2013.83, 2018.37, 2064.55, 2076.02, 2093.63, 2014.5, 2035.99, 2018.82], 'mult_Qmis': [0.0, -0.0, -0.0, 0.78, 11.05, 5.44, 3.54, 7.25, 2.49]}, 'Branch': {'From': [1, 2, 3, 4, 4, 5, 6, 7, 8], 'To': [4, 7, 9, 5, 6, 7, 9, 8, 9], 'Status': [1, 1, 1, 1, 1, 1, 1, 1, 1], 'Sft': [73.27, 111.43, 86.05, 32.2, 42.77, 49.98, 50.0, 63.9, 42.3], 'Stf': [72.63, 111.84, 86.79, 43.0, 44.87, 48.69, 51.77, 64.98, 36.62], 'Slim': [380.0, 250.0, 300.0, 250.0, 250.0, 250.0, 150.0, 250.0, 150.0], 'mult_Sf': [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], 'mult_St': [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]}, 'Gen': {'Genbus': [1, 2, 3

# Appendix

Information about the Jupyter/Python/Container config is listed here.

In [5]:
!jupyter kernelspec list

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Available kernels:
  python3        /opt/views/view/share/jupyter/kernels/python3
  py311-exago    /usr/local/share/jupyter/kernels/py311-exago


In [6]:
!cat /usr/local/share/jupyter/kernels/py311-exago/kernel.json

{
 "argv": [
  "/opt/views/view/bin/python3",
  "-m",
  "ipykernel_launcher",
  "-f",
  "{connection_file}"
 ],
 "display_name": "ExaGO",
 "language": "python",
 "metadata": {
  "debugger": true
 }
}

In [7]:
!jupyter kernelspec list

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Available kernels:
  python3               /opt/views/view/share/jupyter/kernels/python3
  py311-exago           /usr/local/share/jupyter/kernels/py311-exago
  py311-mpi4py-exago    /usr/local/share/jupyter/kernels/py311-mpi4py-exago


In [5]:
!cat /usr/local/share/jupyter/kernels/py311-exago/kernel.json

{
 "argv": [
  "/opt/views/view/bin/python3",
  "-m",
  "ipykernel_launcher",
  "-f",
  "{connection_file}"
 ],
 "display_name": "ExaGO",
 "language": "python",
 "metadata": {
  "debugger": true
 }
}

In [6]:
!pip3 list

Package                  Version
------------------------ ---------
anyio                    4.0.0
argon2-cffi              21.3.0
argon2-cffi-bindings     21.2.0
arrow                    1.2.3
asttokens                2.4.0
async-lru                1.0.3
attrs                    23.1.0
Babel                    2.12.1
backcall                 0.2.0
beautifulsoup4           4.12.2
bleach                   6.0.0
certifi                  2023.7.22
cffi                     1.15.1
charset-normalizer       3.3.0
comm                     0.1.4
debugpy                  1.6.7
decorator                5.1.1
defusedxml               0.7.1
editables                0.3
executing                1.2.0
fastjsonschema           2.16.3
fqdn                     1.5.1
gevent                   23.7.0
greenlet                 2.0.2
hatch-jupyter-builder    0.8.3
hatchling                1.18.0
idna                     3.4
iniconfig                2.0.0
ipykernel                6.23.1
ipython                

That's odd - ExaGO can import and run, but doesn't show up in pip. That's because we installed using Spack, and ExaGO is available directly through the `PYTHONPATH` environment variable.

This is configured through `entrypoint.sh`, which is the last thing that happens in the configured `~/.bashrc`:

In [7]:
!cat ~/.bashrc | tail -n 10

            fi; \
        fi`'
    local lightblue='\[\033\]'
    local removecolor='\[\033\]'
    PS1="${userpart} ${lightblue}\w ${gitbranch}${removecolor}\$ "
    unset -f __bash_prompt
}
__bash_prompt
export PROMPT_DIRTRIM=4
source /entrypoint.sh


In [8]:
!cat /entrypoint.sh

#!/bin/sh
. /opt/spack-environment/activate.sh
exec "$@"


In [9]:
!cat /opt/spack-environment/activate.sh

export SPACK_ENV=/opt/spack-environment;
export SPACK_ENV_VIEW=default;
alias despacktivate='spack env deactivate';
export ACLOCAL_PATH=/opt/views/view/share/aclocal;
export CMAKE_PREFIX_PATH=/opt/views/view;
export C_INCLUDE_PATH=/opt/views/view/include;
export JUPYTERLAB_DIR=/opt/views/view/share/jupyter/lab;
export JUPYTER_PATH=/opt/views/view/share/jupyter;
export LIBRARY_PATH=/opt/views/view/lib;
export MANPATH=/opt/views/view/share/man:/opt/views/view/man:;
export MPICC=/opt/views/view/bin/mpicc;
export MPICXX=/opt/views/view/bin/mpic++;
export MPIF77=/opt/views/view/bin/mpif77;
export MPIF90=/opt/views/view/bin/mpif90;
export PATH=/opt/views/view/bin:/opt/spack/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin;
export PETSC_ARCH="";
export PETSC_DIR=/opt/views/view;
export PKG_CONFIG_PATH=/opt/views/view/lib/pkgconfig:/opt/views/view/share/pkgconfig:/opt/views/view/lib64/pkgconfig;
export PYTHONPATH=/opt/views/view/lib/python3.11/site-packages;


In [10]:
!echo $PYTHONPATH

/opt/views/view/lib/python3.11/site-packages:/opt/views/view/lib/python3.11/site-packages


In [11]:
!ls -al /opt/views/view/lib/python3.11/site-packages

total 864
drwxr-xr-x 193 root root 12288 Nov  6 20:19 .
drwxr-xr-x  39 root root  4096 Nov  6 20:19 ..
drwxr-xr-x   2 root root  4096 Nov  6 20:19 Babel-2.12.1.dist-info
drwxr-xr-x  11 root root  4096 Nov  6 20:19 IPython
drwxr-xr-x   2 root root  4096 Nov  6 20:19 Jinja2-3.0.3.dist-info
drwxr-xr-x   2 root root  4096 Nov  6 20:19 MarkupSafe-2.1.3.dist-info
drwxr-xr-x   2 root root  4096 Nov  6 20:19 PyYAML-6.0.dist-info
drwxr-xr-x   2 root root  4096 Nov  6 20:19 Pygments-2.16.1.dist-info
lrwxrwxrwx   1 root root   139 Nov  6 20:19 README.txt -> /opt/software/linux-ubuntu22.04-x86_64_v3/gcc-11.4.0/python-3.11.6-ybsi7tlnssr3m4h3eqrzq6tyxdcsr2n7/lib/python3.11/site-packages/README.txt
drwxr-xr-x   2 root root  4096 Nov  6 20:19 Send2Trash-1.8.0.dist-info
drwxr-xr-x   2 root root  4096 Nov  6 20:19 __pycache__
drwxr-xr-x   3 root root  4096 Nov  6 20:19 _argon2_cffi_bindings
lrwxrwxrwx   1 root root   175 Nov  6 20:19 _cffi_backend.cpython-311-x86_64-linux-gnu.so -> /opt/software/linux-u

Here you can see the full list of spack configured python packages, including exago.

## Some info about this container

In [12]:
!cat /etc/os-release

PRETTY_NAME="Debian GNU/Linux 12 (bookworm)"
NAME="Debian GNU/Linux"
VERSION_ID="12"
VERSION="12 (bookworm)"
VERSION_CODENAME=bookworm
ID=debian
HOME_URL="https://www.debian.org/"
SUPPORT_URL="https://www.debian.org/support"
BUG_REPORT_URL="https://bugs.debian.org/"


### Previous version of `csv_to_json` (more modularized)

In [14]:
import csv 
import json
import time
import numbers

def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []

    #count the number of lines in the csv file
    lines = 0
    with open(csvFilePath, encoding='utf-8') as csvf: 
        lines = sum(1 for _ in csvf)
    csvf.close()
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's reader
        csvReader = csv.reader(csvf) 

        current_row = 0
        temp = {}
        keys = True
        row = next(csvReader)
        current_row += 1

        while True:
            #set non-number rows as keys in the dictionary
            if keys:
                for y in range(0, len(row)):
                    item = row[y]
                    if item in temp.keys():
                        item += "-2"
                        row[y] = item
                    temp[item] = []
                keys = False
                current_keys = row

            #check if there's another row
            if current_row < lines:
                row = next(csvReader)
                current_row += 1
            else:
                print(temp)
                jsonArray = temp
                #convert python jsonArray to JSON String and write to file
                with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
                    jsonString = json.dumps(jsonArray, indent=4)
                    jsonf.write(jsonString)
                return

            #add values to the current keys until a non-number row
            while not keys:  
                try:
                    #check for the non-number Fuel value
                    if row[2].strip() in ["COAL", "WIND", "SOLAR", "NG", "NUCLEAR", "HYDRO", "UNDEFINED"]:
                        for j in range(0, len(row)):
                            if j != 2:
                                row[j] = float(row[j])
                            else:
                                row[j] = row[2].strip()
                    else:
                        #convert numbers to floats
                        row = [float(i) for i in row] #if this fails, go back to keys

                    #add values to corresponding keys
                    for x in range(0, len(current_keys)):
                        temp[current_keys[x]].append(row[x])

                    #check if there's another row
                    if current_row < lines:
                        row = next(csvReader)
                        current_row += 1
                    else:
                        print(temp)
                        jsonArray = temp
                        #convert python jsonArray to JSON String and write to file
                        with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
                            jsonString = json.dumps(jsonArray, indent=4)
                            jsonf.write(jsonString)
                        return
                except ValueError:
                    #back to keys loop
                    keys = True

#if cell 2 is run, then ./ is the top level of the ExaGO dir
csvFilePath = r'./tmp.output'
jsonFilePath = r'./data.json'

csv_to_json(csvFilePath, jsonFilePath)

print(f"data converted from {csvFilePath} to {jsonFilePath}")

{'nbus': [9.0], 'ngen': [3.0], 'nbranch': [9.0], 'baseMVA': [100.0], 'Bus': [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0], 'Pd': [0.0, 0.0, 0.0, 0.0, 75.0, 90.0, 0.0, 100.0, 0.0], 'Qd': [0.0, 0.0, 0.0, 0.0, 50.0, 30.0, 0.0, 35.0, 0.0], 'Vm': [1.04, 1.02, 1.02, 1.03, 1.01, 1.02, 1.03, 1.02, 1.03], 'Va': [0.0, 4.54, 2.9, -2.19, -3.38, -4.26, 0.75, -1.72, 0.21], 'mult_Pmis': [2064.25, 2013.83, 2018.37, 2064.55, 2076.02, 2093.63, 2014.5, 2035.99, 2018.82], 'mult_Qmis': [0.0, -0.0, -0.0, 0.78, 11.05, 5.44, 3.54, 7.25, 2.49], 'From': [1.0, 2.0, 3.0, 4.0, 4.0, 5.0, 6.0, 7.0, 8.0], 'To': [4.0, 7.0, 9.0, 5.0, 6.0, 7.0, 9.0, 8.0, 9.0], 'Status': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'Sft': [73.27, 111.43, 86.05, 32.2, 42.77, 49.98, 50.0, 63.9, 42.3], 'Stf': [72.63, 111.84, 86.79, 43.0, 44.87, 48.69, 51.77, 64.98, 36.62], 'Slim': [380.0, 250.0, 300.0, 250.0, 250.0, 250.0, 150.0, 250.0, 150.0], 'mult_Sf': [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], 'mult_St': [-0.0, -0.0, -0.0